# This module shows how my GGNN can be run (train\test loop etc.) and shows some initial results, which outperform the baselines in Baselines.ipynb

In [ ]:
import csv
import torch as tt
import time
import os
import numpy as np
import random
from tqdm.notebook import tqdm

from python.utils import CustomTripletLoss
from python.data_loader import GraphDataset
from python.model import GGNN

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
CUDA = True

## Preparing the datasets:

In [ ]:
def load_dataset(directory, filename, batch_size, shuffle, targets, 
                 hidden_size=20, max_nodes=300, edge_types=3, annotation_size=20, target_edge_type=1):
    """
    Load a .json file into memory, create a GraphDataset out of it and return a DataLoader for it
    :param directory:       the directory from which to load the file
    :param filename:        the name of the file
    :param batch_size:      batch size used to initialize the DataLoader
    :param shuffle:         if True, shuffle the graphs on each pass
    :param targets:         Can be either "generate", "generateOnPass", or a key to the json dictionary
                            from which to load the targets
                            "generate": generate targets once and keep them this way (validation)
                            "generateOnPass": generate new targets at each epoch (training)
    :param hidden_size:     the size of node embedding in GGNN that will be used on this dataset
    :param max_nodes:       maximum number of nodes per graph
    :param edge_types:      number of different edge-types. Does not include the edges added to
                            the undirected graph
    :param annotation_size: the size of annotations (initial embedddings) for each node
    :param target_edge_type:the type of edge that is to be predicted
    :return:                a dataloader object
    """
    full_path = os.path.join(directory, filename) # full path to the file
    print("Loading data from %s" % full_path)
    with open(full_path, 'r') as f:
        data = json.load(f)
    dataset =  GraphDataset(data,
                            hidden_size=hidden_size,
                            max_nodes=max_nodes,
                            edge_types=edge_types,
                            annotation_size=annotation_size,
                            targets=targets,
                            target_edge_type=target_edge_type)
    return DataLoader(dataset, shuffle=shuffle, batch_size=batch_size, num_workers=4)

In [ ]:
DIR = '../data/graphs/newMethod/'
test_data = loader.load(DIR, 'test.json', batch_size=10, shuffle=False, targets="targets_1")
train_data = loader.load("train.json", batch_size=10, shuffle=True, targets="generateOnPass")
val_data = loader.load('valid.json', batch_size=10, shuffle=False, targets="generate")

## Epoch\training loops

In [ ]:
def run_epoch(model, data, epoch, is_training):
    """
    Run a given model for one epoch on the given dataset and return the loss and the accuracy
    :param data:         a DataLoader object that works on a GraphDataset
    :param epoch:        epoch id (for logging purposes)
    :param is_training:  whether to train or just evaluate
    :return:             mean loss, mean accuracy
    """
    if CUDA:
        model.cuda()

    if is_training:
        model.train()
        print("Epoch %d. Training" % epoch)
    else:
        model.eval()
        print("Epoch %d. Evaluating" % epoch)

    total_loss = 0
    total_acc = 0
    step = 0
    batches = tqdm(data)

    for adj_matrix, features, src, mask in batches:
        step += 1

        optimizer.zero_grad()
        model.zero_grad()

        batch_size = adj_matrix.shape[0]
        option_size = adj_matrix.shape[1]

        # TODO: move these view functions in the GraphDataset
        adj_matrix = adj_matrix.view(-1, adj_matrix.shape[2], adj_matrix.shape[3]).float()
        src = src.view(-1).long()
        features = features.view(-1, features.shape[2], features.shape[3]).float()

        if CUDA:  # move to CUDA, if possible
            mask = mask.cuda()
            features = features.cuda()
            adj_matrix = adj_matrix.cuda()
            src = src.cuda()

        distances = model(features, adj_matrix, src, batch_size, option_size)

        loss, acc = criterion(distances, mask)
        total_acc += acc.cpu().data.numpy().item()
        total_loss += loss.cpu().data.numpy().item()

        batches.set_description("Acc: s=%f, t=%f. Loss: s=%f, t=%f" % (acc, accuracy / step, loss, total_loss / step))

        if is_training:
            loss.backward(retain_graph=True)
            tt.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

    return total_loss/step, accuracy/step

In [ ]:
def train(model, epochs, patience, train_data, val_data, best_model_file):
    """
    Train a given model for a given number of epochs. Use early stopping with given patience.
    :param model:     a GGNN model
    :param epochs:    maximum number of epochs to run the model for
    :param patience:  patience value to use for early stopping
    :param train_data:training dataset
    :apram val_data:  validation dataset
    :param best_model_file: file to save the best model to
    """

    best_val_loss, best_val_loss_epoch = float("inf"), 0
    for epoch in range(epochs):
        train_loss = run_epoch(model, train_data, epoch, True)
        val_loss = run_epoch(model, val_data, epoch, False)

        if val_loss < best_val_loss:
            tt.save(model.state_dict(), best_model_file)
            print("Best epoch so far. Saving to '%s')" % (best_model_file))
            best_val_loss = val_loss
            best_val_loss_epoch = epoch
        elif epoch - best_val_loss_epoch >= patience:  # early stopping
            print("Early Stopping after %i epochs." % patience)
            break

## Creating and running the model

In [ ]:
model = GGNN(hidden_size = 20,
             annotation_size = 20,
             edge_types=3,
             max_nodes=200,
             timesteps=6)
criterion = CustomTripletLoss(margin=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
train(model, epochs=20, patience=3, train_data=train_data, val_data=val_data, best_model_file = "best.model")

In [ ]:
model.load_state_dict("best.model")

In [ ]:
loss, acc = run_epoch(model, test_data, 1, False)
print(acc)